In [1]:
# from Database import Database
# import sys
# # import multiprocessing as mp
# import pandas as pd
# from collections import Counter
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# from scipy.stats.distributions import chi2
# from bioinfokit import analys, visuz
# from scipy.stats import fisher_exact
# import time
# from scipy.special import factorial


# from fisher import pvalue_npy
# from scipy.stats import chi2_contingency
# from scipy.stats import uniform, randint

from Database import Database
import pandas as pd
from collections import Counter
# Python program to create
# sparse matrix using csr_matrix()
# Import required package
import numpy as np
from scipy.sparse import csr_matrix
# from matplotlib import pyplot as plt

In [2]:
path_db = "D:/Hanze_Groningen/STAGE/db_laatste_copy.db" #"/groups/umcg-wijmenga/tmp01/projects/lude_vici_2021/rawdata/cancer_data/new_db/copydb_L.db"  # /groups/umcg-wijmenga/tmp01/projects/lude_vici_2021/rawdata/cancer_data/new_db/copydatabase_C.db
# Database connection
db = Database(path_db)

df = pd.read_sql('''SELECT project.cancer, sum_dosage_GT.donor_project_ID, 
                        sum_dosage_GT.donor_ID, sum_dosage_GT.snp_ID, sum_dosage_GT.GT, sum_dosage_GT.GT2,
                        snp.chr, snp.pos_start, snp.pos_end , snp.DNase, snp.TFBS, snp.UCNE,
                        snp.in_transcript, snp.in_coding, snp.in_exon, snp.before_gene, snp.after_gene, snp.eQTL, snp.close_eQTL_3000
                FROM project, sum_dosage_GT, snp 
                WHERE sum_dosage_GT.snp_ID=snp.ID AND 
                            sum_dosage_GT.donor_project_ID = project.ID AND 
                            (sum_dosage_GT.GT2 = 1 OR sum_dosage_GT.GT2 = 2) AND 
                            sum_dosage_GT.total_read_count_sum >= 33 ;''', db.mydb_connection)
df

,cancer,donor_project_ID,donor_ID,snp_ID,GT,GT2,chr,pos_start,pos_end,DNase,TFBS,UCNE,in_transcript,in_coding,in_exon,before_gene,after_gene,eQTL,close_eQTL_3000
0,Head and neck,1,1,2,1,1,10,105977264,105977264,0,0,0,1,1,0,1,0,0,0
1,Head and neck,1,1,13,1,1,2,220235789,220235789,0,0,0,0,0,0,0,0,0,0
2,Head and neck,1,1,17,1,1,15,25954365,25954365,1,0,0,1,1,0,0,0,0,0
3,Head and neck,1,1,23,1,1,5,117513900,117513900,0,0,0,0,0,0,0,0,0,0
4,Head and neck,1,1,31,1,1,8,55363870,55363870,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9982581,Mesenchymal,43,2996,21812248,1,1,17,46607840,46607840,1,1,0,1,1,1,0,0,0,0
9982582,Mesenchymal,43,2996,21812249,1,1,15,55919246,55919246,0,0,0,1,1,1,0,0,0,0
9982583,Mesenchymal,43,2996,21812253,1,1,17,42926715,42926715,1,1,0,1,1,1,1,0,0,0
9982584,Mesenchymal,43,2996,21812255,1,1,14,105177455,105177455,0,1,0,1,1,1,0,0,0,0


In [3]:
df_breast = df[df['cancer'] == 'Breast']
df_nbreast = df[df['cancer'] != 'Breast']


In [4]:
region = pd.read_csv("D:/Hanze_Groningen/STAGE/analyse/new/Region_2000_ALL_both_0_TESTS.tsv", sep='\t')
region.sort_values('counts_breast')

,chr,start_region,stop_region,counts_breast,counts_nonbreast,filter_snps_b,filter_snps_nb,counts_breast_plus,counts_nonbreast_plus,X2,p_value_X2_self,log10_p_value_X2_self,p_value_X2,log2_fc,p_value_F
0,chr1,1.0,2000,0.0,0.0,0.000000,0.000000,0.003497,0.000512,0.019939,8.877064e-01,0.051731,1.000000e+00,5.541732,1.000000e+00
959492,chr15,20110001.0,20112000,0.0,12.0,0.000000,0.006148,0.003497,12.000512,1.746184,1.863572e-01,0.729654,3.716918e-01,-8.974029,3.831421e-01
959491,chr15,20108001.0,20110000,0.0,8.0,0.000000,0.004098,0.003497,8.000512,1.161662,2.811212e-01,0.551106,5.817416e-01,-8.389098,6.066981e-01
959490,chr15,20106001.0,20108000,0.0,9.0,0.000000,0.004611,0.003497,9.000512,1.307792,2.527948e-01,0.597232,5.174437e-01,-8.559012,6.139357e-01
959487,chr15,20100001.0,20102000,0.0,4.0,0.000000,0.002049,0.003497,4.000512,0.577143,4.474342e-01,0.349271,9.902140e-01,-7.389190,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674558,chr3,178936001.0,178938000,18.0,17.0,0.062937,0.008709,18.003497,17.000512,46.962053,7.237452e-12,11.140414,2.951659e-11,2.853565,2.824559e-08
1019571,chr10,37736001.0,37738000,19.0,8.0,0.066434,0.004098,19.003497,8.000512,80.409043,3.043990e-19,18.516557,2.566781e-18,4.018967,4.951561e-12
1175022,chr17,7576001.0,7578000,26.0,144.0,0.090909,0.073770,26.003497,144.000512,0.979742,3.222626e-01,0.491790,3.665663e-01,0.301570,2.835720e-01
674566,chr3,178952001.0,178954000,33.0,28.0,0.115385,0.014344,33.003497,28.000512,93.527486,4.005987e-22,21.397290,7.465314e-22,3.008032,3.874678e-15


In [6]:
region_sel_b = df_breast[(df_breast['chr'] == '17') & (df_breast['pos_start'] >= 7578001) & (df_breast['pos_start'] <= 7580000)]
print(len(region_sel_b))

region_sel_nb = df_nbreast[(df_nbreast['chr'] == '17') & (df_nbreast['pos_start'] >= 7578001) & (df_nbreast['pos_start'] <= 7580000)]
print(len(region_sel_nb))

34
162


In [7]:
region.sort_values('counts_nonbreast')

,chr,start_region,stop_region,counts_breast,counts_nonbreast,filter_snps_b,filter_snps_nb,counts_breast_plus,counts_nonbreast_plus,X2,p_value_X2_self,log10_p_value_X2_self,p_value_X2,log2_fc,p_value_F
0,chr1,1.0,2000,0.0,0.0,0.000000,0.000000,0.003497,0.000512,0.019939,8.877064e-01,0.051731,1.000000e+00,5.541732,1.000000e+00
951297,chr15,3720001.0,3722000,0.0,0.0,0.000000,0.000000,0.003497,0.000512,0.019939,8.877064e-01,0.051731,1.000000e+00,5.541732,1.000000e+00
951298,chr15,3722001.0,3724000,0.0,0.0,0.000000,0.000000,0.003497,0.000512,0.019939,8.877064e-01,0.051731,1.000000e+00,5.541732,1.000000e+00
951299,chr15,3724001.0,3726000,0.0,0.0,0.000000,0.000000,0.003497,0.000512,0.019939,8.877064e-01,0.051731,1.000000e+00,5.541732,1.000000e+00
951300,chr15,3726001.0,3728000,0.0,0.0,0.000000,0.000000,0.003497,0.000512,0.019939,8.877064e-01,0.051731,1.000000e+00,5.541732,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306405,chr11,1016001.0,1018000,0.0,239.0,0.000000,0.122439,0.003497,239.000512,34.907956,3.456656e-09,8.461344,7.359589e-10,-13.289875,1.387644e-15
333968,chr11,56142001.0,56144000,0.0,255.0,0.000000,0.130635,0.003497,255.000512,37.244508,1.042080e-09,8.982099,1.623715e-10,-13.383362,1.123818e-16
1181893,chr17,21318001.0,21320000,0.0,269.0,0.000000,0.137807,0.003497,269.000512,39.288889,3.655154e-10,9.437094,4.249336e-11,-13.460470,1.440485e-17
276602,chr7,100550001.0,100552000,0.0,271.0,0.000000,0.138832,0.003497,271.000512,39.580935,3.147409e-10,9.502047,3.503771e-11,-13.471157,8.849395e-18


In [8]:
region_sel_b = df_breast[(df_breast['chr'] == '11') & (df_breast['pos_start'] >= 48386001) & (df_breast['pos_start'] <= 48388000)]
print(len(region_sel_b))

region_sel_nb = df_nbreast[(df_nbreast['chr'] == '11') & (df_nbreast['pos_start'] >= 48386001) & (df_nbreast['pos_start'] <= 48388000)]
print(len(region_sel_nb))

1
284


In [9]:
df_breast.columns


Index(['cancer', 'donor_project_ID', 'donor_ID', 'snp_ID', 'GT', 'GT2', 'chr',
       'pos_start', 'pos_end', 'DNase', 'TFBS', 'UCNE', 'in_transcript',
       'in_coding', 'in_exon', 'before_gene', 'after_gene', 'eQTL',
       'close_eQTL_3000'],
      dtype='object')

In [ ]:
"D:/Hanze_Groningen/STAGE/R/select_inf/vs/filter/gedaan/breast.tsv"
"D:/Hanze_Groningen/STAGE/R/select_inf/vs/filter/gedaan/coding_breast.tsv"
"D:/Hanze_Groningen/STAGE/R/select_inf/vs/filter/gedaan/coding_nonbreast.tsv"
"D:/Hanze_Groningen/STAGE/R/select_inf/vs/filter/gedaan/ncbreast.tsv"
"D:/Hanze_Groningen/STAGE/R/select_inf/vs/filter/gedaan/ncnonbreast.tsv"
"D:/Hanze_Groningen/STAGE/R/select_inf/vs/filter/gedaan/nonbreast.tsv"

In [ ]:
df1 = pd.read_csv("D:/Hanze_Groningen/STAGE/R/select_inf/vs/filter/gedaan/breast.tsv", sep='\t', header=['index', 'chr', 'start', 'stop'])
df2 = pd.read_csv("D:/Hanze_Groningen/STAGE/R/select_inf/vs/filter/gedaan/nonbreast.tsv", sep='\t', header=['index', 'chr', 'start', 'stop'])

df3 = pd.read_csv("D:/Hanze_Groningen/STAGE/R/select_inf/vs/filter/gedaan/coding_breast.tsv", sep='\t', header=['index', 'chr', 'start', 'stop'])
df4 = pd.read_csv("D:/Hanze_Groningen/STAGE/R/select_inf/vs/filter/gedaan/coding_nonbreast.tsv", sep='\t', header=['index', 'chr', 'start', 'stop'])

df5 = pd.read_csv("D:/Hanze_Groningen/STAGE/R/select_inf/vs/filter/gedaan/ncbreast.tsv", sep='\t', header=['index', 'chr', 'start', 'stop'])
df6 = pd.read_csv("D:/Hanze_Groningen/STAGE/R/select_inf/vs/filter/gedaan/ncnonbreast.tsv", sep='\t', header=['index', 'chr', 'start', 'stop'])


In [ ]:
df1

In [10]:
df_breast

,cancer,donor_project_ID,donor_ID,snp_ID,GT,GT2,chr,pos_start,pos_end,DNase,TFBS,UCNE,in_transcript,in_coding,in_exon,before_gene,after_gene,eQTL,close_eQTL_3000
1743820,Breast,25,1305,1325012,1,1,3,178952085,178952085,0,1,0,1,1,1,0,0,0,0
1743821,Breast,25,1305,1922329,1,1,7,50288406,50288406,0,0,0,0,0,0,0,0,0,0
1743822,Breast,25,1305,1979425,1,1,9,104350785,104350785,0,0,0,1,1,0,0,0,0,0
1743823,Breast,25,1305,2129052,1,1,18,73873870,73873870,0,0,0,0,0,0,0,0,0,0
1743824,Breast,25,1305,3656796,1,1,1,9042655,9042655,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2314135,Breast,28,2057,8740869,1,1,10,49931522,49931522,1,1,0,1,1,1,0,0,0,1
2314136,Breast,28,2057,8740870,1,1,17,33772618,33772618,0,1,0,1,1,1,0,0,0,0
2314137,Breast,28,2057,8740871,1,1,7,100187289,100187289,0,1,0,1,1,1,1,0,0,1
2314138,Breast,28,2057,8740873,1,1,20,31463822,31463822,0,1,0,0,0,0,0,0,0,0


In [11]:
print(len(set(df['snp_ID'])))
print('UCNE ' + str(len(set(df[df['UCNE'] == 1]['snp_ID']))))
print('UCNE ' + str(len(set(df[df['UCNE'] == 0]['snp_ID']))))
print()
print('TFBS ' + str(len(set(df[df['TFBS'] == 1]['snp_ID']))))
print('TFBS ' + str(len(set(df[df['TFBS'] == 0]['snp_ID']))))
print()
print('DNase ' + str(len(set(df[df['DNase'] == 1]['snp_ID']))))
print('DNase ' + str(len(set(df[df['DNase'] == 0]['snp_ID']))))
print()
print('in_transcript ' + str(len(set(df[df['in_transcript'] == 1]['snp_ID']))))
print('in_transcript ' + str(len(set(df[df['in_transcript'] == 0]['snp_ID']))))
print()
print('in_exon ' + str(len(set(df[df['in_exon'] == 1]['snp_ID']))))
print('in_exon ' + str(len(set(df[df['in_exon'] == 0]['snp_ID']))))
print()
print('in_coding ' + str(len(set(df[df['in_coding'] == 1]['snp_ID']))))
print('in_coding ' + str(len(set(df[df['in_coding'] == 0]['snp_ID']))))
print()
print('before_gene ' + str(len(set(df[df['before_gene'] == 1]['snp_ID']))))
print('before_gene ' + str(len(set(df[df['before_gene'] == 0]['snp_ID']))))
print()
print('after_gene ' + str(len(set(df[df['after_gene'] == 1]['snp_ID']))))
print('after_gene ' + str(len(set(df[df['after_gene'] == 0]['snp_ID']))))
print()
print('eQTL ' + str(len(set(df[df['eQTL'] == 1]['snp_ID']))))
print('eQTL ' + str(len(set(df[df['eQTL'] == 0]['snp_ID']))))
print()
print('close_eQTL_3000 ' + str(len(set(df[df['close_eQTL_3000'] == 1]['snp_ID']))))
print('close_eQTL_3000 ' + str(len(set(df[df['close_eQTL_3000'] == 0]['snp_ID']))))
print()


9638808
UCNE 5117
UCNE 9633691

TFBS 2328383
TFBS 7310425

DNase 1355697
DNase 8283111

in_transcript 3707807
in_transcript 5931001

in_exon 297867
in_exon 9340941

in_coding 2951776
in_coding 6687032

before_gene 159469
before_gene 9479339

after_gene 159060
after_gene 9479748

eQTL 2633
eQTL 9636175

close_eQTL_3000 226880
close_eQTL_3000 9411928



In [12]:
print(len(set(df_breast['snp_ID'])))
print('UCNE ' + str(len(set(df_breast[df_breast['UCNE'] == 1]['snp_ID']))))
print('UCNE ' + str(len(set(df_breast[df_breast['UCNE'] == 0]['snp_ID']))))
print()
print('TFBS ' + str(len(set(df_breast[df_breast['TFBS'] == 1]['snp_ID']))))
print('TFBS ' + str(len(set(df_breast[df_breast['TFBS'] == 0]['snp_ID']))))
print()
print('DNase ' + str(len(set(df_breast[df_breast['DNase'] == 1]['snp_ID']))))
print('DNase ' + str(len(set(df_breast[df_breast['DNase'] == 0]['snp_ID']))))
print()
print('in_transcript ' + str(len(set(df_breast[df_breast['in_transcript'] == 1]['snp_ID']))))
print('in_transcript ' + str(len(set(df_breast[df_breast['in_transcript'] == 0]['snp_ID']))))
print()
print('in_exon ' + str(len(set(df_breast[df_breast['in_exon'] == 1]['snp_ID']))))
print('in_exon ' + str(len(set(df_breast[df_breast['in_exon'] == 0]['snp_ID']))))
print()
print('in_coding ' + str(len(set(df_breast[df_breast['in_coding'] == 1]['snp_ID']))))
print('in_coding ' + str(len(set(df_breast[df_breast['in_coding'] == 0]['snp_ID']))))
print()
print('before_gene ' + str(len(set(df_breast[df_breast['before_gene'] == 1]['snp_ID']))))
print('before_gene ' + str(len(set(df_breast[df_breast['before_gene'] == 0]['snp_ID']))))
print()
print('after_gene ' + str(len(set(df_breast[df_breast['after_gene'] == 1]['snp_ID']))))
print('after_gene ' + str(len(set(df_breast[df_breast['after_gene'] == 0]['snp_ID']))))
print()
print('eQTL ' + str(len(set(df_breast[df_breast['eQTL'] == 1]['snp_ID']))))
print('eQTL ' + str(len(set(df_breast[df_breast['eQTL'] == 0]['snp_ID']))))
print()
print('close_eQTL_3000 ' + str(len(set(df_breast[df_breast['close_eQTL_3000'] == 1]['snp_ID']))))
print('close_eQTL_3000 ' + str(len(set(df_breast[df_breast['close_eQTL_3000'] == 0]['snp_ID']))))
print()


570048
UCNE 317
UCNE 569731

TFBS 149194
TFBS 420854

DNase 91495
DNase 478553

in_transcript 230085
in_transcript 339963

in_exon 17923
in_exon 552125

in_coding 182943
in_coding 387105

before_gene 11238
before_gene 558810

after_gene 11288
after_gene 558760

eQTL 0
eQTL 570048

close_eQTL_3000 15636
close_eQTL_3000 554412



In [13]:
print(len(set(df_nbreast['snp_ID'])))
print('UCNE ' + str(len(set(df_nbreast[df_nbreast['UCNE'] == 1]['snp_ID']))))
print('UCNE ' + str(len(set(df_nbreast[df_nbreast['UCNE'] == 0]['snp_ID']))))
print()
print('TFBS ' + str(len(set(df_nbreast[df_nbreast['TFBS'] == 1]['snp_ID']))))
print('TFBS ' + str(len(set(df_nbreast[df_nbreast['TFBS'] == 0]['snp_ID']))))
print()
print('DNase ' + str(len(set(df_nbreast[df_nbreast['DNase'] == 1]['snp_ID']))))
print('DNase ' + str(len(set(df_nbreast[df_nbreast['DNase'] == 0]['snp_ID']))))
print()
print('in_transcript ' + str(len(set(df_nbreast[df_nbreast['in_transcript'] == 1]['snp_ID']))))
print('in_transcript ' + str(len(set(df_nbreast[df_nbreast['in_transcript'] == 0]['snp_ID']))))
print()
print('in_exon ' + str(len(set(df_nbreast[df_nbreast['in_exon'] == 1]['snp_ID']))))
print('in_exon ' + str(len(set(df_nbreast[df_nbreast['in_exon'] == 0]['snp_ID']))))
print()
print('in_coding ' + str(len(set(df_nbreast[df_nbreast['in_coding'] == 1]['snp_ID']))))
print('in_coding ' + str(len(set(df_nbreast[df_nbreast['in_coding'] == 0]['snp_ID']))))
print()
print('before_gene ' + str(len(set(df_nbreast[df_nbreast['before_gene'] == 1]['snp_ID']))))
print('before_gene ' + str(len(set(df_nbreast[df_nbreast['before_gene'] == 0]['snp_ID']))))
print()
print('after_gene ' + str(len(set(df_nbreast[df_nbreast['after_gene'] == 1]['snp_ID']))))
print('after_gene ' + str(len(set(df_nbreast[df_nbreast['after_gene'] == 0]['snp_ID']))))
print()
print('eQTL ' + str(len(set(df_nbreast[df_nbreast['eQTL'] == 1]['snp_ID']))))
print('eQTL ' + str(len(set(df_nbreast[df_nbreast['eQTL'] == 0]['snp_ID']))))
print()
print('close_eQTL_3000 ' + str(len(set(df_nbreast[df_nbreast['close_eQTL_3000'] == 1]['snp_ID']))))
print('close_eQTL_3000 ' + str(len(set(df_nbreast[df_nbreast['close_eQTL_3000'] == 0]['snp_ID']))))
print()


9070639
UCNE 4800
UCNE 9065839

TFBS 2179644
TFBS 6890995

DNase 1264456
DNase 7806183

in_transcript 3478430
in_transcript 5592209

in_exon 280081
in_exon 8790558

in_coding 2769400
in_coding 6301239

before_gene 148276
before_gene 8922363

after_gene 147825
after_gene 8922814

eQTL 2633
eQTL 9068006

close_eQTL_3000 211302
close_eQTL_3000 8859337

